# Solving the Simplest Case of the Helmholtz Equation

We consider the time-dependent Helmholtz equation:

$$
i \, \mathrm{d}t \, u = - \Delta u
$$

with homogeneous Dirichlet boundary conditions:

$$
u(x,t) = 0, \quad \forall x \in \partial \Omega,\; t > 0
$$

and initial condition:

$$
u(x,0) = 1, \quad \forall x \in \Omega
$$

The exact solution is given by:

$$
u(x,t) = \sin(\pi x_1) \sin(\pi x_2) e^{-i 2\pi^2 t}
$$




## Verification of the Exact Solution

We solve the equation

$$
i\, \partial_t u = -\Delta u,
$$

with the proposed exact solution

$$
u(x,t) = \sin(\pi x_1)\sin(\pi x_2)\, e^{-i 2\pi^2 t}.
$$

---

### 1. Time Derivative

We compute

$$
\partial_t u(x,t) = \sin(\pi x_1)\sin(\pi x_2) \cdot \frac{d}{dt} \left( e^{-i 2\pi^2 t} \right)
= -i 2\pi^2 \sin(\pi x_1)\sin(\pi x_2) \, e^{-i 2\pi^2 t}.
$$

Multiplying by \( i \):

$$
i\, \partial_t u = i \cdot (-i 2\pi^2) \sin(\pi x_1)\sin(\pi x_2) \, e^{-i 2\pi^2 t}
= 2\pi^2 \sin(\pi x_1)\sin(\pi x_2) \, e^{-i 2\pi^2 t}.
$$

---

### 2. Laplacian

We compute

$$
\Delta u = \frac{\partial^2 u}{\partial x_1^2} + \frac{\partial^2 u}{\partial x_2^2}.
$$

Now

$$
\frac{\partial^2 u}{\partial x_1^2} = -\pi^2 \sin(\pi x_1) \sin(\pi x_2) \, e^{-i 2\pi^2 t}, \quad
\frac{\partial^2 u}{\partial x_2^2} = -\pi^2 \sin(\pi x_1) \sin(\pi x_2) \, e^{-i 2\pi^2 t}.
$$

Thus

$$
\Delta u = -2\pi^2 \sin(\pi x_1) \sin(\pi x_2) \, e^{-i 2\pi^2 t},
$$

and so

$$
- \Delta u = 2\pi^2 \sin(\pi x_1) \sin(\pi x_2) \, e^{-i 2\pi^2 t}.
$$

---

### ✅ Conclusion

We have:

$$
i\, \partial_t u = -\Delta u,
$$

so the exact solution satisfies the Helmholtz equation.


In [11]:


# imports
import numpy as np
import dolfinx
from dolfinx import fem, io
from dolfinx.fem.petsc import LinearProblem
import ufl
from ufl import conj
from petsc4py import PETSc
from slepc4py import SLEPc
from mpi4py import MPI


Model parameters + mesh and functionspace construction

In [11]:
# create mesh
n = 1000
nx, ny = n, n
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, nx, ny, dolfinx.mesh.CellType.triangle)



# timestep parameters
T = 1.0  # total time
nt = 100  # number of time steps
dt = T / nt  # time step size


# create function space
V = fem.functionspace(mesh, ("Lagrange", 1))

Analytical solution: 

$ u(x,t) = \sin(\pi x_1) \sin(\pi x_2) * \exp(- i 2 \pi^2 t) $

watch out here is a factor of $i/(i+2 \pi^2 dt)$ in the exact solution because when computing the error some constant value remains which is $2\pi^2 dt$ but I'm not getting it seems still wrong


In [12]:
# exact solution is 
u_exact = dolfinx.fem.Function(V, dtype=np.complex128)

u_exact.interpolate(lambda x: np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]) * \
                                np.exp(-1j * 2 * np.pi**2 * dt)* (1j/(1j+2 * np.pi**2*dt)))
u_analytic = dolfinx.fem.Function(V, dtype=np.complex128)
u_analytic.interpolate(lambda x: np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]) * np.exp(-1j * 2 * np.pi**2 * dt))



Assemble the System:

In [13]:
# test and trial functions
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
# rhs is initial condition f = u0 = sin(pi x_1) sin(pi x_2)
f =  dolfinx.fem.Function(V, dtype=np.complex128)
f.interpolate(lambda x: np.sin(np.pi * x[0]) * np.sin(np.pi * x[1]))

# bilinear + linear form
a = (1j / dt) * ufl.inner(u, v) * ufl.dx + ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (1j / dt) * ufl.inner(f, v) * ufl.dx


Enforce the boundary conditions:

In [14]:

# define boundary condition
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)

boundary_facets = dolfinx.mesh.exterior_facet_indices(mesh.topology)
boundary_dofs = dolfinx.fem.locate_dofs_topological(V, mesh.topology.dim-1, boundary_facets)
zero = fem.Constant(mesh, PETSc.ScalarType(0.0 + 0.0j))
bc = fem.dirichletbc(zero, boundary_dofs, V)


Solve the problem:

In [15]:

# solve problem
problem = LinearProblem(a, L, bcs=[bc])
uh = problem.solve()


Post-process the solution/ error analysis:

In [16]:
# compute error
error = uh - u_exact

error_analytic = uh - u_analytic

# compute L2 norm of error
l2_error = np.sqrt(fem.assemble_scalar(fem.form(ufl.inner(error, error) * ufl.dx)))
# take real part as imaginary should be 0 anyways
l2_error = (l2_error.real)

l2_error_analytic = np.sqrt(fem.assemble_scalar(fem.form(ufl.inner(error_analytic, error_analytic) * ufl.dx)))
l2_error_analytic = (l2_error_analytic.real)

print(f"L2 norm of error: {l2_error:.12f}")
print(f"L2 norm of error with analytic solution: {l2_error_analytic:.12f}")

L2 norm of error: 0.096670634196
L2 norm of error with analytic solution: 0.193262603592


Use Helmholtz Class from problem_1.py

In [8]:
import simon.problem_1 as problem_1
import importlib
importlib.reload(problem_1)

<module 'problem_1' from '/workspaces/AG_FEM_PINN/problem_1.py'>

In [9]:
from simon.problem_1 import HelmholtzProblem

test = HelmholtzProblem(16,16,1, 100)

Problem with h =  0.0625 , dt =  0.01 , l2 corrected =  0.09692531351953142 , l2 =  0.5086752898742148


In [ ]:
# create new object
